In [ ]:
!pip install datasets peft transformers bitsnbytes evaluate accelerate

  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nv

In [ ]:
pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.8 MB/s eta 0:00:00


In [ ]:
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.9 MB/s eta 0:00:00


In [ ]:
import torch
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import DatasetDict
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import numpy as np
import evaluate
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu



In [ ]:
model_path = 'CohereForAI/aya-101'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("CohereForAI/aya-101")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
training = load_from_disk('/content/drive/MyDrive/Thesis/data/train')

In [ ]:
validation = load_from_disk('/content/drive/MyDrive/Thesis/data/validation')

In [ ]:
testing = load_from_disk('/content/drive/MyDrive/Thesis/data/eval')

In [ ]:
tokenized_dataset = DatasetDict({
    "train": training,
    "validation": validation,
    "test": testing,
})

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
})

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_path, load_in_8bit=True, device_map="auto")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

model-00001-of-00011.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00011.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00011.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

model-00009-of-00011.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00010-of-00011.safetensors:   0%|          | 0.00/2.99G [00:00<?, ?B/s]

model-00011-of-00011.safetensors:   0%|          | 0.00/4.10G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/11 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [ ]:
model

T5ForConditionalGeneration(
  (shared): Embedding(250112, 4096)
  (encoder): T5Stack(
    (embed_tokens): Embedding(250112, 4096)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
              (k): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
              (v): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
              (o): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
              (relative_attention_bias): Embedding(32, 64)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear8bitLt(in_features=4096, out_features=10240, bias=False)
              (wi_1): Linear8bitLt(in_features=4096,

In [ ]:
lora_config = LoraConfig(
 r=8,
 lora_alpha=32,
 target_modules="all-linear",
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)

In [ ]:
model = prepare_model_for_kbit_training(model)

In [ ]:
model = get_peft_model(model, lora_config)

In [ ]:
model.print_trainable_parameters()

trainable params: 35,389,440 || all params: 12,956,446,720 || trainable%: 0.2731


In [ ]:
label_pad_token_id = -100

In [ ]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [ ]:
output_dir="/content/drive/MyDrive/Thesis/model_aya_checkpoints"

In [ ]:
drive_path = '/content/drive/MyDrive/Thesis/model_aya'
model.save_pretrained(drive_path)
tokenizer.save_pretrained(drive_path)

('/content/drive/MyDrive/Thesis/model_aya/tokenizer_config.json',
 '/content/drive/MyDrive/Thesis/model_aya/special_tokens_map.json',
 '/content/drive/MyDrive/Thesis/model_aya/tokenizer.json')

In [ ]:
model.base_model.save_pretrained(drive_path)

In [ ]:
import wandb
wandb.login()

In [ ]:
import os

In [ ]:
os.environ["WANDB_PROJECT"]="thesis-lora-aya-101"

In [ ]:
os.environ["WANDB_LOG_MODEL"] = "checkpoint"

In [ ]:
os.environ["WANDB_PROJECT"]

'thesis-lora-aya-101'

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
	auto_find_batch_size=True,
    learning_rate=1e-4,
    num_train_epochs=2,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="epoch",
    report_to="wandb",
)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"]
)

In [ ]:
trainer.train()
wandb.finish()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


wandb: Currently logged in as: tanmay-k (team-tk). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
100,2.192100
200,2.038200
300,2.032000
400,2.031500
500,1.960800
600,2.012300
700,1.968600
800,1.958000
900,1.923600
1000,1.923300


wandb: Adding directory to artifact (/content/drive/MyDrive/Thesis/model_aya_checkpoints/checkpoint-2500)... Done. 1.2s
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
wandb: Adding directory to artifact (/content/drive/MyDrive/Thesis/model_aya_checkpoints/checkpoint-5000)... Done. 1.1s
wandb: Adding directory

train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/grad_norm,▅▃▃▆▂▅▂▇▁▄▃▄▆▃▂▃▄▄▆▃▃▃▅▅█▃▄▆▆▆▅▇▅▆▇▄▆▆█▃
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,█▆▆▆▅▅▄▄▄▄▄▄▄▄▄▄▃▃▃▃▂▂▂▁▂▂▂▂▂▁▂▂▁▁▂▂▂▂▁▁
total_flos,1.46620217819136e+18
train/epoch,2.0
train/global_step,5000
train/grad_norm,1.47034
train/learning_rate,0.0
train/loss,1.7515


In [ ]:
def generate(example, word):
  prompt = """example: {}

word: {}

provide a dictionary definition for the word based on the example.

definition: """
  input_ids = tokenizer(prompt.format(example, word), return_tensors="pt", truncation=True, max_length = 512).input_ids.cuda()
  with torch.inference_mode():
    outputs = outputs = model.generate(
        input_ids=input_ids,
        max_length=50,
        num_beams=4,
        early_stopping=True,
        num_return_sequences=1
    )
    #outputs = model.generate(input_ids=input_ids, max_new_tokens=150, do_sample=True, top_p=0.9)
  print(f"input word: {word}\n{'---'* 20}\nDefinition:")
  return tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

In [ ]:
example = "institutions are rejecting warnings that house prices could curpe because low interest rates are likely to be held far into next year ."

word = "curpe"

In [ ]:
print(generate(example, word))

input word: curpe
------------------------------------------------------------
Definition:
( of a price, quantity, etc. ) fall in value


In [ ]:
example = "this ocular emergency requires intravenous antibiotics , piptolaryngology , and ophthalmologic consultation ."

word = "piptolaryngology"

In [ ]:
print(generate(example, word))

input word: piptolaryngology
------------------------------------------------------------
Definition:
the surgical treatment of infections of the throat and laryngeal mucosa.


In [ ]:
test_dataset = load_from_disk("/content/drive/MyDrive/Thesis/data/eval/").with_format("torch")

In [ ]:
from tqdm import tqdm

In [ ]:
def evaluate_peft_model(sample, max_target_length=50):
    outputs = model.generate(
        input_ids=sample["input_ids"].unsqueeze(0).cuda(),
        max_length=max_target_length,
        num_beams=4,
        early_stopping=True, 
        num_return_sequences=1
    )
    prediction = tokenizer.decode(outputs[0].cpu().numpy(), skip_special_tokens=True)
    labels = np.where(sample['labels'] != -100, sample['labels'], tokenizer.pad_token_id)
    labels = tokenizer.decode(labels, skip_special_tokens=True)
    return prediction, labels

In [ ]:
predictions, references = [] , []
for sample in tqdm(test_dataset):
    p,l = evaluate_peft_model(sample)
    predictions.append(p)
    references.append([l])

  0%|          | 0/200 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
100%|██████████| 200/200 [15:30<00:00,  4.65s/it]


In [ ]:
metric = evaluate.load("rouge")

In [ ]:
rouge = metric.compute(predictions=predictions, references=references)

In [ ]:
rouge

{'rouge1': 0.28948584626467244,
 'rouge2': 0.10815320047403071,
 'rougeL': 0.26508837282096387,
 'rougeLsum': 0.26542220719368037}

In [ ]:
smoother = SmoothingFunction().method4

In [ ]:
bleu_score = corpus_bleu(references, predictions, smoothing_function=smoother, weights=(0.25, 0.25, 0.25, 0.25), auto_reweigh=True)

print(f"Adjusted BLEU score: {bleu_score * 100:.2f}%")

Adjusted BLEU score: 25.28%
